In [2]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shirl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_json("C:\\Users\\shirl\\Desktop\\MovieReactionDS_small.json")

#CONTROLEREN OP NULL WAARDES
df.isna().sum()

#LEESTEKENS 
df["Gecleand"] = df['input'].str.replace('[^\w\s]','')

#De kolom `Gecleand` alles in kleine letters zetten
df['Gecleand'] = df['Gecleand'].str.lower()
#print(df)

#stopwoorden eruit halen
stop = stopwords.words('english')
df['Gecleand'] = df['Gecleand'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
test = le.fit_transform(df['output'])
print(test)

                                                input    output  \
0   Supercraptastic slasher fare, which feels over...  negative   
1   Even by the lowered standards of '80s slasher ...  negative   
2   Foolish hikers go camping in the Utah mountain...  negative   
3   Rented 3 bad movies to watch with my friends i...  negative   
4   I was pulled into this movie early on, much to...  negative   
5   The prey has an interesting history, unless yo...  negative   
6   All the criticisms of this movie are quite val...  negative   
7   I walked into a book store in Brentwood, Tenne...  negative   
8   Thank God I didn't buy this movie myself! I bo...  negative   
9   I saw this movie with my girlfriend. It was a ...  negative   
10  If anyone tells you this picture is just terri...  negative   
11  Are you kidding me? This is quite possibly the...  negative   
12  In addition to the fact that this is just an a...  negative   
13  I wanted to like this movie. I really, really ...  negativ

<ipython-input-6-6e9c4b1f9beb>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Gecleand"] = df['input'].str.replace('[^\w\s]','')


In [14]:
data = df['Gecleand'].str.split(" ", expand=False)
print(data)

from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report



X_train, X_test, y_train, y_test = train_test_split(
    data, df['output'], test_size=0.2)

w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

words = set(w2v_model.wv.index_to_key)
print(words)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                        for ls in X_test])


X_train_vect_avg = []
for v in X_train_vect:
    print(v)
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

lr_clf = LogisticRegression()
lr_clf.fit(X_train_vect_avg, y_train.values)

print('Test Accuracy: ', lr_clf.score(X_test_vect_avg, y_test.values))

print(classification_report(y_test,
                            lr_clf.predict(X_test_vect_avg)))

0     [supercraptastic, slasher, fare, feels, overly...
1     [even, lowered, standards, 80s, slasher, movie...
2     [foolish, hikers, go, camping, utah, mountains...
3     [rented, 3, bad, movies, watch, friends, dorm,...
4     [pulled, movie, early, much, surprise, hadnt, ...
5     [prey, interesting, history, unless, remember,...
6     [criticisms, movie, quite, valid, pretty, bori...
7     [walked, book, store, brentwood, tennessee, go...
8     [thank, god, didnt, buy, movie, borrowed, frie...
9     [saw, movie, girlfriend, total, disaster, real...
10    [anyone, tells, picture, terrific, probably, s...
11    [kidding, quite, possibly, worst, amateur, mov...
12    [addition, fact, abysmally, made, film, imagin...
13    [wanted, like, movie, really, really, excited,...
14    [understanding, people, praising, movie, didnt...
15    [tale, upperclasses, getting, comeuppance, wal...
16    [filmmaker, stayed, true, accurate, account, s...
17    [able, see, preview, movie, uclas, prescre

<ipython-input-14-8721dd1d96b9>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
<ipython-input-14-8721dd1d96b9>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
C:\Users\shirl\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t